In [1]:
dicts = [
    {'type':'manned', 'zone':'two'},
    {'type':'unmanned', 'zone':'one'},
    
]

In [2]:
new_list = []
for rec in dicts:
    if rec['type']=='manned':
        new_list.append(rec)

In [3]:
new_list

[{'type': 'manned', 'zone': 'two'}]

In [5]:
new_list = [rec for rec in dicts if rec['type']=='manned']
new_list

[{'type': 'manned', 'zone': 'two'}]